In [47]:
##### This code is from Updated Testing Reddit - No Con- bias (Fictitious Play)-01092022
##### This code replace the big real data network with small synthetic network
import scipy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
import pandas as pd
import scipy.io
import collections
import sys
import scipy.optimize
import os
import os.path
from game import Game, GameResult, exportGameResult
from utils import *

    # Fixed initial condition + memeory = 50

## Mathmatic Functions

### 1. Import Network

In [48]:
network_name = 'manual'
# network_name = 'reddit'
# network_name = 'karate'

In [49]:
network = import_network(network_name)
G, s, n = network.init()

Process the Network Data

In [50]:
# the Laplacian matrix
L = scipy.sparse.csgraph.laplacian(G, normed=False)
# A = (I + L)^(-1) - Stanford paper theory
A = np.linalg.inv(np.identity(n) + L)
nxG = nx.from_numpy_array(G)

### 2. Equilibrium & Polarization 
$$P(z) = z ^T * z $$


## 5. Innate Op and Game

# Fictitious Play Start !


In [51]:
# Calculate network polarization
calculate_polarization(s, n, A, L)

Innate_polarization:	0.41022891110361115
Equi_polarization:	0.04411213470472514
Difference:		-0.366116776398886


In [52]:
# Game Parameters
game_rounds = 20
memory = 10
k = 2
experiment = 1

In [53]:
# configure the game
game = Game(s, A, L, k)
# run the game
result = game.run(game_rounds, memory)

Nodes, opinions
[1, 4] (1, 1)
Nodes, opinions
[2, 3] (0.5, 0.5)
min_history [((2, 3), (0.5, 0.5))]
--------------------
Game 1
----------
min_history [((2, 3), (0.5, 0.5))]
max_history [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0]
fla_min_fre [1.]
Max_por 0.09181344162282332
Max_por 0.1244223977280837
Max_por 0.1244223977280837
all_por [0.009 0.042 0.042 0.092 0.    0.    0.    0.    0.    0.    0.    0.
 0.056 0.07  0.124 0.016 0.    0.    0.    0.    0.    0.    0.    0.
 0.018 0.112 0.075 0.046 0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.   ]
column - best action: 14
Maximizer found its target 2 agent: [0, 4] op: (1, 0)
fre_max at spot 0.5
M
[[ 0.169  0.146 -0.031 -0.063 -0.22 ]]
M
[[-0.091  0.063 -0.137 -0.074  0.24 ]]
champion:  [2, 3] [0.412 0.618] 0.08465731554022113
Minimizer finds its target agents: [2, 3]
fla_min_fre: [0.5 0.5]
Not Reached Nash Equilibrium at Equi_Min = 0.08465731554022113 and Equi_Max = 0.12442239772

In [54]:
exportGameResult(game, result, k, memory, experiment)